Добейтесь точности распознавания жанров музыки не менее 79% стабильно на последних эпохах обучения. 

Используйте разбивку 900 записей на обучающую выборку и 100 на проверочную.

### Классификация музыкальных жанров


In [ ]:
from google.colab import files
from tensorflow.keras.utils import to_categorical
import os # Работа с папками и файлами
import librosa # Параметризация аудио
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Conv2D, LSTM
from sklearn.model_selection import train_test_split # Разбиение на обучающую и проверочную выборку
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q '/content/drive/My Drive/Базы/genres.zip' # распаковываем архив на google диске на локальный диск google colaboratory 

genres = os.listdir('genres') # получаем список папок в распакованной папке

# Проверяем выгруженные папки
!ls genres 
# И одну из папок
!ls genres/blues

replace genres/blues/blues.00000.au? [y]es, [n]o, [A]ll, [N]one, [r]ename: blues  classical  country  disco  hiphop  jazz	metal  pop  reggae  rock
blues.00000.au	blues.00020.au	blues.00040.au	blues.00060.au	blues.00080.au
blues.00001.au	blues.00021.au	blues.00041.au	blues.00061.au	blues.00081.au
blues.00002.au	blues.00022.au	blues.00042.au	blues.00062.au	blues.00082.au
blues.00003.au	blues.00023.au	blues.00043.au	blues.00063.au	blues.00083.au
blues.00004.au	blues.00024.au	blues.00044.au	blues.00064.au	blues.00084.au
blues.00005.au	blues.00025.au	blues.00045.au	blues.00065.au	blues.00085.au
blues.00006.au	blues.00026.au	blues.00046.au	blues.00066.au	blues.00086.au
blues.00007.au	blues.00027.au	blues.00047.au	blues.00067.au	blues.00087.au
blues.00008.au	blues.00028.au	blues.00048.au	blues.00068.au	blues.00088.au
blues.00009.au	blues.00029.au	blues.00049.au	blues.00069.au	blues.00089.au
blues.00010.au	blues.00030.au	blues.00050.au	blues.00070.au	blues.00090.au
blues.00011.au	blues.00031.a

In [ ]:
# Функция параметризации аудио
def get_features(y, sr):
  # Получаем различные параметры аудио
  chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr) # Частота цветности (по умолчанию 12 баков цветности)
  mfcc = librosa.feature.mfcc(y=y, sr=sr) # Мел кепстральные коэффициенты (по умолчанию 20)

  rmse = np.mean(librosa.feature.rms(y=y)) # Среднеквадратичная амплитуда
  spec_cent = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)) # Среднее спектрального центроида
  spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)) # Среднее ширины полосы частот
  rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)) # Среднее спектрального спада частоты
  zcr = np.mean(librosa.feature.zero_crossing_rate(y)) # Среднее частота пересечения нуля звукового временного ряда
  

  # Добавляем все параметры в один список
  out = []
  out.append(rmse) # добавляем среднеквадратичную амплитуду
  out.append(spec_cent) # добавляем спектральный центроид
  out.append(spec_bw) # добавляем ширину полосы частот
  out.append(rolloff) # добавляем спектральный спад частоты
  out.append(zcr) # добавляем пересечение нуля
 

  # Добавляем среднее всех Мел спектральных коэффициентов (20 коэффициентов)
  for e in mfcc:
    out.append(np.mean(e))
  
  # Добавляем среднее всех частот цветности (12 коэффициентов)
  for e in chroma_stft:
    out.append(np.mean(e))
  
  # Возвращаем получившийся список размерностью (37,) 
  return out


In [ ]:
import time # Подключаем модуль time для подсчёта времени на обработку одного жанра

# Формируем обучающую выборку
# Создаём пустые листы
X_train = []
Y_train = []

# Запоминаем время старта формирования выборки
curr_time = time.time()

# Проходим по всем жарнам
for i in range(len(genres)):
  g = genres[i] # Берём текущий жанр
  # Проходим по файлам папки, соответствующей текущему жанру
  for filename in os.listdir(f'./genres/{g}'):
    # Получаем имя песни
    songname = f'./genres/{g}/{filename}'
    # Загружаем в y аудиосигнал
    # Используем первые 30 секунд аудио
    y, sr = librosa.load(songname, mono=True, duration=30) # y - массив данных временного ряда аудио, sr - частота дискретизации временного ряда
    # Превращаем сигнал в параметризованные данные
    out = get_features(y, sr)
    
    
    # Добавляем строку в X_train
    X_train.append(out)
    # Добавляем в Y_train номер жанра в формате ohe
    Y_train.append(to_categorical(i, len(genres)))

  # Выводим информацию о готовности обработки базы
  print("Жанр ", g, " готов -> ", round(time.time() - curr_time), "c", sep="")
  curr_time = time.time()

# Превращаем обучающую выборку на numpy массивы
X_train = np.array(X_train)
Y_train = np.array(Y_train)


Жанр country готов -> 51c
Жанр pop готов -> 49c
Жанр reggae готов -> 51c
Жанр blues готов -> 50c
Жанр disco готов -> 49c
Жанр jazz готов -> 49c
Жанр classical готов -> 50c
Жанр rock готов -> 45c
Жанр hiphop готов -> 49c
Жанр metal готов -> 49c


In [ ]:
# Создаем backup обучающей выборки
X_train_backup = X_train.copy()
Y_train_backup = Y_train.copy()


In [ ]:
print(len(X_train[5]))
print(len(X_train))

37
991


In [ ]:
X_train[77]

array([ 8.71645361e-02,  1.32783139e+03,  1.63297451e+03,  2.75602500e+03,
        6.10873101e-02, -2.17141983e+02,  1.36159256e+02, -1.03507509e+01,
        5.13835869e+01,  2.61628222e+00,  1.51244001e+01, -6.66066468e-01,
        1.22880259e+01, -6.22800922e+00,  7.79273224e+00, -5.41524839e+00,
        3.62371922e+00, -2.80927062e+00, -3.51482153e-01, -5.53418040e-01,
        3.70173484e-01, -2.24874830e+00, -2.49541402e+00,  4.18859339e+00,
        3.42443496e-01,  2.06485897e-01,  3.75383794e-01,  5.76249003e-01,
        3.35114568e-01,  2.69194245e-01,  2.25021616e-01,  3.35900366e-01,
        3.39530081e-01,  3.79021823e-01,  5.80321133e-01,  3.17055941e-01,
        2.87962735e-01])

In [ ]:
# Выводим номера классов, для проверки правильности заполнения
# И номера классов идут последовательно крупными блоками
y_train_class = np.argmax(Y_train, axis=1)
print(y_train_class)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [ ]:
# Выводим размеры обучающей выборки
print(X_train.shape)
print(Y_train.shape)
print(y_train_class.shape)

(991, 37)
(991, 10)
(991,)


In [ ]:
# Создаём scaler экземпляр класса StandardScaler() для нормировки данных
scaler = StandardScaler()
# Нормируем X_train
X_train = scaler.fit_transform(X_train)

In [ ]:
#Проверяем, что X_train нормировался
print(X_train[0])

[-2.05660075e-01  2.56097268e-01  8.92895517e-01  5.15233377e-01
 -3.95311616e-01  4.31105650e-01 -4.88212104e-01  7.60898307e-01
 -3.46326430e-01  1.22302514e+00 -2.46614564e-01  4.96785673e-01
 -1.43042274e+00  5.48799758e-01 -1.21568558e+00 -3.28471330e-01
 -2.37109617e-01  8.59002736e-01 -1.70372827e+00 -1.00134792e-01
 -1.24367181e+00 -1.31451484e+00 -1.59583965e+00 -2.19719408e+00
 -1.66445097e+00  6.06977099e-01  6.33920061e-01  2.40765245e-01
 -3.81026925e-01  2.11625574e-01 -6.04393269e-01 -1.02283421e+00
 -6.96794032e-01 -2.08100877e-03  1.26617246e+00  6.97025297e-01
  3.17788628e-01]


In [ ]:
# Разделяем выборку на обучающую и проверочную
# Для проверочной используем 10 % примеров, так как база маленькая
X_train, X_test, Y_train, Y_test = train_test_split(X_train, y_train_class, test_size = 0.1)

In [ ]:
# Выводим размеры обучающей и проверочной выборок для проверки
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(891, 37)
(891,)
(100, 37)
(100,)


### Создаем нейронку

In [ ]:
import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

chkpt = ModelCheckpoint('model.h5', 
                        monitor='val_accuracy', 
                        verbose=1, 
                        save_best_only=True, 
                        mode='auto')
lrPlato = ReduceLROnPlateau(monitor='val_mae', 
                            factor=0.2, 
                            patience=5, 
                            verbose=0, 
                            mode='auto', 
                            min_delta=0.0001, 
                            cooldown=20, 
                            min_lr=1e-5) 

In [ ]:
# Указываем, какие индексы данных во входных векторах брать для обучения
# Делаем это для того, чтобы можно было экспериментировать
# И обучать не на всех колонках данных, а на части
indexes = range(0,37)

# Создаем полносвязную сеть
model = Sequential()
model.add(Dense(256, activation='elu', input_shape=(len(indexes),)))
model.add(Dense(128,activation='elu'))
model.add(Dropout(0.25))
model.add(Dense(64,activation='elu'))
model.add(Dense(32,activation='elu'))
model.add(BatchNormalization())
# В выходном слое количество нейронов равно количеству классов(жанров) и активация softmax
model.add(Dense(len(genres), activation = 'softmax'))


In [ ]:
# Компилируем сеть
model.compile(optimizer=RMSprop(lr=1e-6),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(X_train,
                    Y_train,
                    epochs = 10,
                    batch_size = 32,
                    validation_data = (X_test, Y_test),
                    callbacks = [chkpt])

Epoch 1/10
19/28 [===================>..........] - ETA: 0s - loss: 0.0055 - accuracy: 0.9967     
Epoch 00001: val_accuracy did not improve from 0.80000
28/28 [==============================] - 2s 10ms/step - loss: 0.0162 - accuracy: 0.9944 - val_loss: 2.3590 - val_accuracy: 0.8000
Epoch 2/10
20/28 [====================>.........] - ETA: 0s - loss: 0.0230 - accuracy: 0.9937    
Epoch 00002: val_accuracy did not improve from 0.80000
28/28 [==============================] - 0s 4ms/step - loss: 0.0270 - accuracy: 0.9921 - val_loss: 2.3560 - val_accuracy: 0.8000
Epoch 3/10
20/28 [====================>.........] - ETA: 0s - loss: 0.0168 - accuracy: 0.9953
Epoch 00003: val_accuracy did not improve from 0.80000
28/28 [==============================] - 0s 4ms/step - loss: 0.0181 - accuracy: 0.9944 - val_loss: 2.3501 - val_accuracy: 0.8000
Epoch 4/10
21/28 [=====================>........] - ETA: 0s - loss: 0.0342 - accuracy: 0.9911    
Epoch 00004: val_accuracy did not improve from 0.80000
28/

In [ ]:
#model.save_weights('model.h5')
model.load_weights('model.h5')